In [40]:
import requests
import pandas as pd

In [41]:
api_key = "api_key"

In [42]:
def fetch_books(query, api_key, max_results=10):
    """
    Fetch book data from Google Books API with pagination.

    Args:
        query (str): Search term or keyword (e.g., "data science").
        api_key (str): Your Google Books API key.
        max_results (int): Total number of books to fetch (max 200).

    Returns:
        list: A list of book details.
    """
    books = []  # List to store book data
    url = "https://www.googleapis.com/books/v1/volumes"
    start_index = 0
    max_per_request = 40  # API's limit per request

    while start_index < max_results:
        # Define parameters for the current request
        params = {
            "q": query,
            "key": api_key,
            "startIndex": start_index,
            "maxResults": min(max_per_request, max_results - start_index),
        }

        # Make API request
        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"Error: {response.status_code} - {response.text}")
            break

        # Parse response
        data = response.json()
        items = data.get("items", [])
        if not items:
            print("No more data to fetch.")
            break

        # Append book data to the list
        for item in items:
            volume_info = item.get("volumeInfo", {})
            sale_info = item.get("saleInfo", {})
            books.append({
                "book_id": item.get("id", ""),
                "search_key": query,
                "book_title": volume_info.get("title", ""),
                "book_subtitle": volume_info.get("subtitle", "NA"),
                "book_authors": ", ".join(volume_info.get("authors", [])),
                "book_publisher": volume_info.get("publisher", ""),                
                "industryIdentifiers": volume_info.get("industryIdentifiers", [{}])[0].get("type", ""),
                "text_readingModes": volume_info.get("readingModes", {}).get("text", False),
                "image_readingModes": volume_info.get("readingModes", {}).get("image", False),
                "categories": ", ".join(volume_info.get("categories", [])),
                "published_year": volume_info.get("publishedDate", "")[:4] if volume_info.get("publishedDate", "") else "",
                "saleability": sale_info.get("saleability", ""),
                "list_price": sale_info.get("listPrice", {}).get("amount", 0.0),
                "currencyCode_listPrice": sale_info.get("listPrice", {}).get("currencyCode", ""),
                "retail_price": sale_info.get("retailPrice", {}).get("amount", 0.0),
                "currencyCode_retailPrice": sale_info.get("retailPrice", {}).get("currencyCode", ""),
                "average_rating": volume_info.get("averageRating", 0.0),
                "ratings_count": volume_info.get("ratingsCount", 0),
                "country": volume_info.get("country", ""),
                "page_count": volume_info.get("pageCount", 0),
                "language": volume_info.get("language", ""),                
                "is_ebook": sale_info.get("isEbook", False),                
            })

        # Increment start index for pagination
        start_index += max_per_request

    return books


In [43]:
"""queries = [
    "html",
    "javascript",
    "statics",
    "artificial intelligence",
    "sql"
]
books_data = []
for query in queries:
        print(f"Fetching books for query: {query}")
        all_books = fetch_books(query, api_key)
        books_data.extend(all_books)
"""
books_data = fetch_books('civics', api_key)

In [44]:
# Convert to a DataFrame for better organization
books_df = pd.DataFrame(books_data)
books_df.head(2)

,book_id,search_key,book_title,book_subtitle,book_authors,book_publisher,industryIdentifiers,text_readingModes,image_readingModes,categories,...,list_price,currencyCode_listPrice,retail_price,currencyCode_retailPrice,average_rating,ratings_count,country,page_count,language,is_ebook
0,EXUWEAAAQBAJ,civics,Excursions 8 History/Civics- (17-18),NA,No Author,HarperCollins,ISBN_13,False,True,Education,...,276.0,INR,193.2,INR,0.0,0,,176,en,True
1,MHM-EAAAQBAJ,civics,Solutions to FOCUS ON History and CIVICS for C...,NA,"I. L. Wanchoo, Arun Kumar, Goyal Brothers Prak...",Goyal Brothers Prakashan,,False,True,Juvenile Nonfiction,...,80.0,INR,80.0,INR,0.0,0,,33,en,True


In [45]:
books_df.loc[2][:]

book_id                                                 JwFjDwAAQBAJ
search_key                                                    civics
book_title                  New Transitions  History and Civics  6
book_subtitle                                                     NA
book_authors                          Anuradha Sud, Shiladitya Ghosh
book_publisher                                Vikas Publishing House
industryIdentifiers                                          ISBN_13
text_readingModes                                              False
image_readingModes                                              True
categories                                            Social Science
published_year                                                      
saleability                                                 FOR_SALE
list_price                                                     584.1
currencyCode_listPrice                                           INR
retail_price                      

In [46]:
books_df.shape

(10, 22)

In [47]:
books_df.fillna('', inplace=True)

In [48]:
#Install MySQL Connector
#!pip install mysql-connector-python

In [49]:
#Connect Jupyter Notebook to MySQL:
import mysql.connector
from mysql.connector import Error

# Define the connection parameters
host = "localhost"        # MySQL server address
user = "root"    # Your MySQL username
password = "D8g11U77"  # Your MySQL password
database = "database1"  # Database name to connect to

# Establish a connection
try:
    # Connect to the MySQL database
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )

    if connection.is_connected():
        print("Connected to MySQL database")
        
    # Create a cursor to execute SQL queries
    cursor = connection.cursor()
      
except Error as e:
    print(f"Error connecting to MySQL: {e}")




Connected to MySQL database


In [50]:
# Create a cursor to execute SQL queries
cursor = connection.cursor()

In [51]:
# Define the table creation SQL query
create_table_query = """
CREATE TABLE IF NOT EXISTS books_table (
    book_id VARCHAR(255) PRIMARY KEY, 
    search_key VARCHAR(255),         
    book_title VARCHAR(255),         
    book_subtitle TEXT,              
    book_authors TEXT,
    book_publisher TEXT,                
    industryIdentifiers TEXT,        
    text_readingModes BOOLEAN,       
    image_readingModes BOOLEAN,
    categories TEXT,
    published_year INT,
    saleability VARCHAR(50),
    list_price DECIMAL(10, 2), 
    currencyCode_listPrice VARCHAR(10), 
    retail_price DECIMAL(10, 2), 
    currencyCode_retailPrice VARCHAR(10),
    average_rating DECIMAL(3, 2) NULL,
    ratings_count INT,
    country VARCHAR(10),
    page_count INT,                   
    language VARCHAR(10),        
    is_ebook BOOLEAN    
)
"""
cursor.execute(create_table_query)
print("Table `books_table` created or already exists.")      
            
         


Table `books_table` created or already exists.


In [52]:
insert_query = """
INSERT INTO books_table (
    book_id, 
    search_key, 
    book_title, 
    book_subtitle, 
    book_authors, 
    book_publisher,     
    industryIdentifiers, 
    text_readingModes, 
    image_readingModes,     
    categories,
    published_year,          
    saleability,      
    list_price, 
    currencyCode_listPrice, 
    retail_price, 
    currencyCode_retailPrice,    
    average_rating,
    ratings_count,     
    country,
    page_count,
    language, 
    is_ebook
) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Execute insertion for all rows
#for index, row in books_df.iterrows():
    #cursor.execute(insert_query, (row['book_id'], row['search_key'], row['book_title'], row['book_subtitle'], row['book_authors'], row['book_publisher'], row['industryIdentifiers'], row['text_readingModes'], row['image_readingModes'], row['categories'], row.get['published_year', ''], row['saleability'], row['list_price'], row['currencyCode_listPrice'], row['retail_price'], row['currencyCode_retailPrice'], row['average_rating'], row['ratings_count'], row['country'], row['page_count'], row['language'], row['is_ebook']))
    
# Convert DataFrame rows to a list of tuples
data_to_insert = books_df.to_records(index=False).tolist()

# Execute insertion for all rows
cursor.executemany(insert_query, data_to_insert)
connection.commit()
print(f"{cursor.rowcount} rows inserted successfully into `books_table`.")     

10 rows inserted successfully into `books_table`.
